In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate 
!pip install -q langchain-community langchain-core

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
nbclassic 0.5.2 requires prometheus-client, which is not installed.
jupyter-server 1.23.4 requires prometheus-client, which is not installed.
databricks-feature-store 0.16.4 requires pyspark<4,>=3.1.2, which is not installed.
mlflow-skinny 2.8.1 requires packaging<24, but you have packaging 24.1 which is incompatible.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
import torch
import pandas as pd
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2024-07-11 06:43:00.828191: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 06:43:01.634995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 06:43:01.635042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 06:43:01.639516: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 06:43:02.030492: I tensorflow/core/platform/cpu_feature_g

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=512,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

hfp = HuggingFacePipeline(pipeline=pipeline_inst)

In [ ]:
template = """Question:Translate the following text into spanish: {text}
OUTPUT:"""
prompt_translate = PromptTemplate.from_template(template)
chain_translate = prompt_translate | hfp


template = """Question: rephrase the following text - {text}
OUTPUT:"""
prompt = PromptTemplate.from_template(template)
chain_rephrase = prompt | hfp


In [ ]:
def translate(hf_pl,language, text):
    template = f"""Question:Translate the following text into {language}: {text}

    TRASLATION: """
    prompt = PromptTemplate.from_template(template)

    chain = prompt | hf_pl

    response = chain.invoke({"question": text})
    
    return response.split('TRASLATION:')[1]




In [ ]:
def rephrase(hf_pl, text):
    template = f"""Question: rephrase the following text - {text}
    REPHRASE: """
    prompt = PromptTemplate.from_template(template)

    chain = prompt | hf_pl

    response = chain.invoke({"question": text})
    
    return response.split('REPHRASE:')[1]

In [ ]:
translate(hfp,'spanish','I went to the coffe next door and order a sandwitch ')

' \n\n    Voy a la café next door y pido un sándwich'

In [ ]:
rephrase(hfp,'I went to the coffe next door and order a sandwitch ')

'  I made my way to a nearby coffee establishment and placed an order for a sandwich.'

# Augmentation

In [ ]:
from datasets import load_dataset
dataset = load_dataset("darrow-ai/LegalLensNLI-SharedTask")

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df = pd.DataFrame(data = dataset)

In [ ]:
rows = []
for row in df.iterrows():
    new_row = [(row[1]['train']['premise']),(row[1]['train']['hypothesis']),(row[1]['train']['legal_act']),(row[1]['train']['label'])]
    rows.append(new_row)

In [ ]:
df_new = pd.DataFrame(rows,columns=['premise','hypothesis','legal_act','label'])

In [ ]:
df_new_tmp = df_new.head(25)

In [ ]:
def handle_bulk(text_lst,chain):
    input = [ {'text':txt} for txt in text_lst ]
    response =chain.batch(input)
    extracted_output = [res.split('OUTPUT:')[1] for res in response]
    return extracted_output


In [ ]:
def generate_text_batch(df,in_col, out_col, chain, batch_size=4):
    genereated_texts = []
    for i in range(0, len(df), batch_size):
        batch = df[in_col].iloc[i:i+batch_size].tolist()
        translated_batch = handle_bulk(batch, chain)
        genereated_texts.extend(translated_batch)
    
    df[out_col] = genereated_texts
    return df


In [ ]:
df_new = generate_text_batch(df_new,'hypothesis','hypothesis_rephrased',chain_rephrase,6)

In [ ]:
df_new = generate_text_batch(df_new,'premise','premise_rephrased',chain_rephrase,6)

In [ ]:
df_new = generate_text_batch(df_new,'premise','premise_translated',chain_translate,6)

In [ ]:
df_new = generate_text_batch(df_new,'hypothesis','hypothesis_translated',chain_translate,6)

In [ ]:
df_new.to_csv('/root/darrow-data-lake/csv/nli_aug_raw.csv',index=False)

In [ ]:
df_new

,premise,hypothesis,legal_act,label,hypothesis_rephrased,premise_rephrased,premise_translated,hypothesis_translated
0,DEFENDANT has reached a settlement in a class ...,Had to visit DEFENDANT a while back for some r...,privacy,Neutral,"Recently, I went to see DEFENDANT to have som...",The class action lawsuit concerning the data ...,EL DEFENDENT ha alcanzado un acuerdo de asent...,Tuví que visitar a DEFENDANTE recientes para ...
1,A class action lawsuit has been certified agai...,"So, at 22, I was into this whole ""collect-and-...",consumer_protection,Entailed,"At 22 years old, I was into the ""collect-and-...",A class action suit has been sanctioned again...,\nLa demanda de acción colectiva ha sido certi...,"Así, a los 22 años, estaba en este tendido de..."
2,"DEFENDANT, an auto parts supplier, has agreed ...",As an employee of the aforementioned auto part...,consumer_protection,Contradict,"As an employee of an auto parts supplier, I h...","DEFENDANT, a business that sells automotive p...","DEFENDENT se ha comprometido por pagar $950,0...",Como empleado de la empresa de suministros de...
3,"DEFENDANT has agreed to pay $400,000 to settle...","Hey, got a call from DEFENDANT a while back, s...",privacy,Contradict,"Hi there, just wanted to let you in on the fa...",The settlement amount for a lawsuit filed by ...,"\nEl DEFENSO ha acordado pagar $400,000 para r...","Hola, me llegí una trucada d'ACUSADO fa algun..."
4,DEFENDANT and other health benefit companies h...,"Just checked my mail, got a letter from DEFEND...",privacy,Neutral,I have just reviewed my emails and found a le...,The health benefit companies including DEFEND...,Los partidos defensor y otras empresas de ben...,Sobre la última hora comprobé mi correo elect...
...,...,...,...,...,...,...,...,...
307,DEFENDANT has reached a settlement in a breach...,Feeling a bit perplexed today. I've been a loy...,consumer_protection,Entailed,"Today, I am slightly bewildered. I have been ...",The defendant in a breach of contract class a...,El DEFENDENT ha llegat a un acord de compromí...,Sintiendo un poco confundido hoy en día. He e...
308,"DEFENDANT, a seafood restaurant operator in Ca...",Had a fantastic seafood dinner at this place l...,privacy,Neutral,"Last night, I had an amazing seafood dinner a...",A seafood restaurant company known as DEFENDA...,"\n\nLa compañía DEFENDANT, operador de restaur...",Tení una increíble cena de marisco anoche en ...
309,Consumers who received promotional text messag...,Hardly ever use my phone for anything other th...,tcpa,Neutral,"Rarely, I use my phone for purposes other tha...",\nIndividuals in the United States who receive...,Consumers quien recibieron mensajes de promoc...,Raramente uso mi teléfono para nada más que l...
310,"DEFENDANT, a restaurant point-of-sale provider...","Upon my daily visits to the local diner, I fre...",privacy,Entailed,During my routine visits to the neighborhood ...,"DEFENDANT, a restaurant point-of-sale provide...","EL DEFENSIÓN, un proveedor de sistemas de pun...",Durante mis visitas diarias al restaurante lo...


### Concat Results 

In [ ]:
df_origin = df_new[['premise','hypothesis','legal_act','label']]
df_origin['augmentation'] = 'origin'

df_translate = df_new[['premise_translated','hypothesis_translated','legal_act','label']].rename(columns={'premise_translated':'premise','hypothesis_translated':'hypothesis'})
df_translate['augmentation'] = 'translation'

df_rephrase = df_new[['premise_rephrased','hypothesis_rephrased','legal_act','label']].rename(columns={'premise_rephrased':'premise','hypothesis_rephrased':'hypothesis'})
df_rephrase['augmentation'] = 'rephrase'

/root/.ipykernel/2531/command-3371427619778138-3529435739:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin['augmentation'] = 'origin'


In [ ]:
pd.concat([df_origin,df_translate,df_rephrase]).to_csv('nli_aug.csv',index=False)